# PyTorch and TensorFlow

一些在学习使用 PyTorch 和 TensorFlow 的时候的笔记。

## 关于维度

我之前一直没理解为什么 PyTorch 里一些 NLP 相关的模块（如 LSTM、GRU）输入一批数据的时候要规定默认数据的维度的第二维是 batch size，而不是把 batch size 放在第一个维度。

例如 GRU 的 input 默认 shape 是 (max_length, batch_size, input_size)，即第一维是每条语句的最大长度（词数量），第二维是批大小，第三位是词嵌入的维度大小。当然，我也可以在传入数据的时候设置参数 `batch_first=True` 这样我就可以传入 (batch_size, max_length, input_size) 的数据了，这样的数据组织方式更符合直观理解。

后来看了 PyTorch Chatbot 的教程才知道，这样做的好处是，如果我想自己在代码里一步一步处理输入序列，每次手动取出一个 time-step，对输入数据取第一个维度的下标的时候，一次就能取到所有 batch 的该下标对应的 time-step。

如果把 batch size 放在第一个维度，那么想取所有 batch 的某一个 time-step 的话，就要用到 Python 的 slice 操作，例如 

In [11]:
import numpy as np
data = np.array([[11, 12, 13],
                 [21, 22, 23],
                 [31, 32, 33]])
print(data[:,1])

[12 22 32]


学习 PyTorch 的 chatbot 教程 中的 attention 的时候，总是对一些输入输出的维度弄不清楚。现在在这里记录一下，假设只看 `forward` 函数，输入的变量是已经 `embedding` 了，在 decoder 中我们是一步一步地处理

```
def forward(self, hidden, encoder_outputs):
    # Calculate the attention weights (energies)
    attn_energies = self.atten(hidden, encoder_outputs)

    # Transpose max_length and batch_size dimensions
    attn_energies_t = attn_energies.t()
    
    # Weights are softmax normalized probability scores (with added dimension)
    attn_energies_t_softmax = F.softmax(attn_energies_t, dim=1)
    atten_weights = attn_energies_t_softmax.unsqueeze(1)

    return atten_weights
```
这里的各个变量的维度应该是这样的：

* hidden.shape: (1, batch_size, hidden_size)
* encoder_outputs.shape: (length, batch_size, embed_size)
* attn_energies.shape: (length, batch_size)
* attn_energies_t.shape: (batch_size, length)
* attn_energies_t_softmax.shape: (batch_size, length)
* atten_weights.shape: (batch_size, 1, length)

在 `return` 的时候把 batch_size 放在第一位的原因是通常会用 `torch.bmm(input, mat2, out=None)` 来把 batch 的 `atten_weights` 和 batch 的 `encoder_outputs` 做 batch matrix-matrix product，这个函数的要求就是 batch_size 是第一个维度。

> If input is a $(b \times n \times m)$ tensor, mat2 is a $(b \times m \times p)$ tensor, out will be a $(b \times n \times p)$ tensor.

因此，在得到了 `atten_weights` 之后，要把 `atten_weights` 应用到 `encoder_outputs` 上时，`encoder_outputs` 也会做一个转置，把 batch_size 放到第一位

```
weighted_sum = torch.bmm(atten_weights, encoder_outputs.transpose(0, 1))
```

此时

* weighted_sum.shape: (batch_size, 1, embed_size)

这时候 `weighted_sum` 的 dimension 就 make sense 了，即 batch_size 个 weighted sum, 每个 weighted sum 的维度是 (1, embed_size)。

当然，如果想用别的方法而不是 `torch.bmm` 来应用 attention 的权重的时候，可以不必把 batch_size 放到第一位。

## 实现通用 Attention 时的坑

我在根据 *Attention Is All You Need* 实现一个通用的 Attention 类的时候，踩到了一个很不起眼的坑。导致在训练模型的时候，刚开始训练没多久，模型的输出以及模型自身的参数就都变成了 `nan`，找了很久很久才找到 bug 在哪。

训练模型如果想加速，就要把多个数据打包成一个 batch 去训练。然后在 NLP 领域，数据长度通常是不同的，就需要 padding，但在某些地方我们不想把这些 padding 的值被考虑进来，就要想办法处理一下。

*Attention Is All You Need* 里面的方法就是在计算 attention 的时候传入一个 mask 参数，这个 mask 就是用来表示哪些值是训练数据里的有效值，哪些是无效值（即 padding）。在计算完 attention energy 之后，本来是直接继续用 softmax 函数就能够得到和为 1 的权重。但传入了 mask 之后，就会把无效值的位置对应的 attention energy 设置为负无穷大，即 `-float('inf')`，然后再用 softmax 函数计算，这时候这些无效位置得到的权重就是 0。

我踩到的坑是，如果在运行时，有某个数据对应的 mask 表示该数据是全都无效的，这时候使用 softmax 函数之前， attention energy 就会是全为负无穷大，然后 softmax 函数计算出来的就全都是 `nan`。

（为什么会有某个数据对应的 mask 表示该数据是全都无效？那这个数据完全没必要存在啊。其实原因是在做 padding 的时候，有些数据结构真的比较复杂，在多个数据组合成 batch 的时候就真的有可能出现一个**全都是** padding 的无用数据。）

In [1]:
import torch
sftm = torch.nn.Softmax(dim=-1)
energy_1 = torch.tensor([1.0, 2.0, 3.0, 4.0])
energy_2 = torch.tensor([1.0, 2.0, 3.0, -float("inf")])
energy_3 = torch.tensor([-float("inf"), -float("inf"), -float("inf"), -float("inf")])
energy = torch.stack([energy_1,energy_2,energy_3])
sf_energy = sftm(energy)
print(sf_energy)

tensor([[0.0321, 0.0871, 0.2369, 0.6439],
        [0.0900, 0.2447, 0.6652, 0.0000],
        [   nan,    nan,    nan,    nan]])


所以要对 softmax 的输出再进行处理一下。比如，可以把所有 `nan` 都填充为 0。这时候可以利用一个特性，那就是 `nan != nan` 会返回 `True`，也就是两个 `nan` 其实不相等。

In [25]:
print(sf_energy != sf_energy)

tensor([[False, False, False, False],
        [False, False, False, False],
        [ True,  True,  True,  True]])


所以我们就可以把 `True` 对应的位置设置为 0 即可。

In [26]:
sf_energy[sf_energy != sf_energy] = 0
print(sf_energy)

tensor([[0.0321, 0.0871, 0.2369, 0.6439],
        [0.0900, 0.2447, 0.6652, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000]])


注意！如果是在模型训练阶段，在反向求导的时候，会有报错
```
RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation
```
必须用 `masked_fill`，`masked_fill_` 也会报错（带下划线尾缀的函数是 inplace 操作）。

In [2]:
nan_mask = sf_energy != sf_energy
sf_energy = sf_energy.masked_fill(nan_mask, 0)
sf_energy

tensor([[0.0321, 0.0871, 0.2369, 0.6439],
        [0.0900, 0.2447, 0.6652, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000]])